In [ ]:
# default_exp decisiontree_prescriber

In [ ]:
%load_ext autoreload
%autoreload 2

# Decision tree prescriber

> This module finds importance weights for NPIs. The rationale is the following. For every region we can adjust a random forest that predicts the impact of a combination of NPIs in a given country. We will measure the impact with five levels (0-4) the represent the categories: Very bad, Bad, Neutral, Good, Very good. This categories are defined with respect to how well they should work in reducing the number of new cases once the interventions are implemented.





In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import MeanShift, estimate_bandwidth

from covid_xprize.standard_predictor import xprize_predictor

from nixtamal_covid import data
import pickle

In [ ]:
#exports
def make_training_df(sim_df, lookup_days):
    ''' Create column with percentage change in the future, given the rows NPI and a number of days in the future'''
    
    df = sim_df.copy()
    df.Date = pd.to_datetime(df.Date)
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    
    df['CumCases'] = df.groupby(['GeoID', 'SimCode'])['PredNewCases'].transform('cumsum')
    df[f'CumCases_next{lookup_days}'] = df.groupby(['GeoID',
                                                    'SimCode'])['CumCases'].transform('shift',
                                                                                        -lookup_days)
    df[f'CumCases_{lookup_days}'] = df.groupby(['GeoID',
                                                'SimCode'])['CumCases'].transform('shift',
                                                                                  lookup_days)
    
    df[f'SmthCases_{lookup_days}'] = (df['CumCases'] -
                                      df[f'CumCases_{lookup_days}']) / lookup_days
    
    df[f'SmthCases_next{lookup_days}'] = (df[f'CumCases_next{lookup_days}'] -
                                          df['CumCases']) / lookup_days
    
    df[f'PctChange_{lookup_days}'] = (df[f'SmthCases_next{lookup_days}'] -
                                      df[f'SmthCases_{lookup_days}']) / (df[f'SmthCases_{lookup_days}'] + 1)
    
    df = df[df[f'CumCases_{lookup_days}'].notna() & df[f'CumCases_next{lookup_days}'].notna()]
    
    return df

In [ ]:
#exports
def train_random_forests(df):
    '''Training data. X is given by the current IPS, new cases per 100k habitants and country data.
    Y is given by the NIPs that give the lowest number of new cases '''
    
    geo_ids = df.GeoID.unique()
    npi_importances = {}
    geoid_rfs = {}
    for geo_id in geo_ids:
        geo_data = df[df.GeoID == geo_id]
        regr = RandomForestRegressor(random_state=0)
        regr.fit(geo_data[xprize_predictor.NPI_COLUMNS], geo_data[f'PctChange_{lookup_days}'])
        npi_importances.update({geo_id: regr.feature_importances_})
        #geoid_rfs.update({geo_id: regr})
    
    npi_importance_df = pd.DataFrame.from_dict(npi_importances, orient='index')
    npi_importance_df.columns = [xprize_predictor.NPI_COLUMNS]
    
    npi_importance_df = npi_importance_df.reset_index()
    npi_importance_df.rename(columns={'index': 'GeoID'}, inplace=True)
    
    return npi_importance_df

In [ ]:
# exports
def make_clusters(npi_importance_df):
    bandwidth = estimate_bandwidth(npi_importance_df.drop(columns='GeoID'), quantile=0.1)
    ms = MeanShift(bandwidth=bandwidth)

    ms.fit(npi_importance_df.drop(columns='GeoID'))
    
    return ms

In [ ]:
load_pickle = False

In [ ]:
if load_pickle:
    with open('simulations_df.pkl', 'rb') as f:
        sim_df = pickle.load(f)
else:
    sim_df = data.load_predictor_simulations()
    with open('simulations_df.pkl', 'wb') as f:
        pickle.dump(sim_df, f)

In [ ]:
lookup_days = 7

In [ ]:
df = make_training_df(sim_df, lookup_days)
del sim_df

In [ ]:
df[(df['GeoID'] == 'Mexico') & (df.SimCode == '332423202324')].plot(y=[f'SmthCases_{lookup_days}',
                                                                       f'SmthCases_next{lookup_days}'])

In [ ]:
df[(df['GeoID'] == 'Mexico') &
   (df.SimCode == '332423202324')].plot(y=f'PctChange_{lookup_days}')

In [ ]:
npi_importance_df = train_random_forests(df)

In [ ]:
npi_importance_df.to_csv('npi_importances.csv', index=False)
npi_importance_df = pd.read_csv('npi_importances.csv')

In [ ]:
npi_imp = npi_importance_df.query('GeoID == "Mexico"').copy()
npi_imp

In [ ]:
npi_imp.drop(columns='GeoID', inplace=True)
npi_imp*npi_imp

In [ ]:
npi_importance_df.describe()

In [ ]:
npi_importance_df.query('GeoID == "Spain"')

In [ ]:
npi_importance_df.query('GeoID == "Mexico"')

In [ ]:
ms = make_clusters(npi_importance_df)

In [ ]:
ms.cluster_centers_

In [ ]:
np.unique(ms.labels_)

In [ ]:
npi_importance_df[ms.labels_ == 7]

In [ ]:
ms.labels_